## Med Data warehouse

### Preprocessing data
  - This notebook is for creating a stuctured database tables with the detected image results

#### Importing necessary libraries


In [1]:
import os
import sys

import pandas as pd

In [2]:
sys.path.append(os.path.abspath('../scripts'))
from database import DbConn, DatabaseConn

#### The results for the detected images

In [3]:
detected_images = pd.read_csv('../data/detection_results.csv')

In [4]:
detected_images.head()

,Image,xmin,ymin,xmax,ymax,confidence,class
0,@lobelia4cosmetics_11340.jpg,253.544296,6.880188,919.716980,1172.334717,0.870472,Supplements
1,@lobelia4cosmetics_11370.jpg,126.841553,9.094360,1199.680908,1230.457886,0.637349,Milk_Formula
2,@lobelia4cosmetics_11381.jpg,187.824158,0.000000,967.715515,1280.000000,0.916270,Supplements
3,@lobelia4cosmetics_11356.jpg,182.825165,0.000000,1027.187500,1280.000000,0.661063,Food_packages
4,@lobelia4cosmetics_11329.jpg,168.980850,24.522741,1006.682129,1159.659790,0.610535,Milk_Formula


In [5]:
detected_images.shape

(2541, 7)

#### Cleaned data from our database

In [6]:
dbconn = DbConn()
db_data = dbconn.read_data('MedData_Transformed')

In [7]:
db_data = pd.DataFrame(db_data, columns=['Product', 'Price', 'Address', 'Tellno', 'Channel_username', 'Message', 'Date', 'Media_path', 'ID'])
db_data.head()

,Product,Price,Address,Tellno,Channel_username,Message,Date,Media_path,ID
0,GLUCERNA \n,3200 birr,Infront of Bole Medhanialem high school \nOpen...,0911562031/0911587703\nAdres,@lobelia4cosmetics,GLUCERNA \nPrice 3200 birr \nTelegram https://...,2024-07-10 12:26:19,data/photos\@lobelia4cosmetics_10013.jpg,10013
1,MIELLE OIL \n,2500 birr,Infront of Bole Medhanialem high school \nOpen...,0911562031/0911587703\nAdres,@lobelia4cosmetics,MIELLE OIL \nPrice 2500 birr \nTelegram https...,2024-07-10 12:27:02,data/photos\@lobelia4cosmetics_10014.jpg,10014
2,now CASTOR OIL 473ML\n,4000 birr,Infront of Bole Medhanialem high school \nOpen...,0911562031/0911587703\nAdres,@lobelia4cosmetics,now CASTOR OIL 473ML\nPrice 4000 birr \nTelegr...,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10015.jpg,10015
3,DAY & NIGHT QUIL \n,2200 birr\n,Infront of Bole Medhanialem high school \nOpen...,0911562031/0911587703\nAdres,@lobelia4cosmetics,DAY & NIGHT QUIL \nPrice 2200 birr\nTelegram h...,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10016.jpg,10016
4,,,,,@lobelia4cosmetics,,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10017.jpg,10017


#### Further preprocessing

In [8]:
db_data['Product'] = db_data['Product'].apply(lambda x: x.strip())
db_data['Address'] = db_data['Address'].apply(lambda x: x.split('\n')[0])
db_data['Tellno'] = db_data['Tellno'].apply(lambda x: x.split('\n')[0])
db_data['Price'] = db_data['Price'].apply(lambda x: x.split(' birr')[0])
# Selecting data that have prices for every product
db_data = db_data[db_data['Price'].str.isnumeric()]

In [9]:
db_data.head()

,Product,Price,Address,Tellno,Channel_username,Message,Date,Media_path,ID
0,GLUCERNA,3200,Infront of Bole Medhanialem high school,0911562031/0911587703,@lobelia4cosmetics,GLUCERNA \nPrice 3200 birr \nTelegram https://...,2024-07-10 12:26:19,data/photos\@lobelia4cosmetics_10013.jpg,10013
1,MIELLE OIL,2500,Infront of Bole Medhanialem high school,0911562031/0911587703,@lobelia4cosmetics,MIELLE OIL \nPrice 2500 birr \nTelegram https...,2024-07-10 12:27:02,data/photos\@lobelia4cosmetics_10014.jpg,10014
2,now CASTOR OIL 473ML,4000,Infront of Bole Medhanialem high school,0911562031/0911587703,@lobelia4cosmetics,now CASTOR OIL 473ML\nPrice 4000 birr \nTelegr...,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10015.jpg,10015
3,DAY & NIGHT QUIL,2200,Infront of Bole Medhanialem high school,0911562031/0911587703,@lobelia4cosmetics,DAY & NIGHT QUIL \nPrice 2200 birr\nTelegram h...,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10016.jpg,10016
5,ENFAMIL AR 354GM,4200,Infront of Bole Medhanialem high school,0911562031/0911587703,@lobelia4cosmetics,ENFAMIL AR 354GM \nPrice 4200 birr \nTelegram ...,2024-07-10 12:27:03,data/photos\@lobelia4cosmetics_10019.jpg,10019


**Observation**
- Even after further cleaning there are some columns that are null and I did not drop these columns because they contain data that could be detected by YOLO

In [10]:
# Extracting ID from the detection results table to create relation with other tables
detected_images['ID'] = detected_images['Image'].apply(lambda x: x.split('_')[1].split('.')[0])
detected_images.loc[:, 'ID'] = detected_images['ID'].astype('int64')
detected_images = detected_images.drop_duplicates(subset=['ID'], keep='first')

In [11]:
# Creating Products tabel
Ids = detected_images['ID'].values
product_col = ['Product', 'Price', 'Address', 'Tellno', 'ID']
Products = db_data[product_col]
Products = Products[Products['ID'].isin(Ids)]
Products.set_index('ID', inplace=True)

In [12]:
# Creating a table that is only related to Telegram
Ids = detected_images['ID'].values
telegram_col = ['Channel_username', 'Message', 'Date', 'Media_path', 'ID']
Telegram = db_data[telegram_col]
Telegram = Telegram[Telegram['ID'].isin(Ids)]
Telegram.set_index('ID', inplace=True)

In [13]:
# This is to select only the images that are detected and are found in the above cleaned tables from the database
Ids = Products.index
detected_images = detected_images[detected_images['ID'].isin(Ids)]
detected_images.set_index('ID', inplace=True)
detected_images.columns = ['Image', 'x_min', 'y_min', 'x_max', 'y_max', 'confidence', 'label']

In [14]:
Products.shape, detected_images.shape, Telegram.shape

((1901, 4), (1901, 7), (1901, 4))

In [15]:
databaseConn = DatabaseConn()
databaseConn.insert_dataframe_data('Products', Products)
databaseConn.insert_dataframe_data('Telegram', Telegram)
databaseConn.insert_dataframe_data('detected_images', detected_images)